# Model Monitoring with Evidently and MLFlow

In [20]:
!which pip
!pip --version
import sys
print(sys.executable)

!{sys.executable} -m pip install -U evidently

/Users/plinphon/HSclasses/M12_MLOps/monitoring-for-modern-ai-and-mlops-plinphon/.venv/bin/pip
pip 25.1.1 from /Users/plinphon/HSclasses/M12_MLOps/monitoring-for-modern-ai-and-mlops-plinphon/.venv/lib/python3.13/site-packages/pip (python 3.13)
/Users/plinphon/HSclasses/M12_MLOps/monitoring-for-modern-ai-and-mlops-plinphon/.venv/bin/python


In [31]:
import evidently
print(dir(evidently))

['BinaryClassification', 'ColumnType', 'DataDefinition', 'Dataset', 'MulticlassClassification', 'Recsys', 'Regression', 'Report', 'Run', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_jupyter_nbextension_paths', '_pydantic_compat', '_registry', '_version', 'compare', 'core', 'descriptors', 'errors', 'legacy', 'llm', 'metrics', 'nbextension', 'presets', 'pydantic_utils', 'sdk', 'tests', 'ui', 'version_info']


In [36]:
%load_ext autoreload
%autoreload 2
!pip install -U evidently

import joblib
import pandas as pd
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from pathlib import Path
from typing import Text, Any, Dict
from sklearn import ensemble, model_selection

from evidently import DataDefinition
from evidently import Report
from evidently.metrics.regression import (
    RegressionQualityMetric,
    RegressionPredictedVsActualScatter,
    RegressionPredictedVsActualPlot,
    RegressionErrorPlot,
    RegressionAbsPercentageErrorPlot,
    RegressionErrorDistribution,
    RegressionErrorNormality,
)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'RegressionPredictedVsActualScatter' from 'evidently.metrics.regression' (/Users/plinphon/HSclasses/M12_MLOps/monitoring-for-modern-ai-and-mlops-plinphon/.venv/lib/python3.13/site-packages/evidently/metrics/regression.py)

In [4]:
!pip list | grep mlflow

mlflow                             3.1.0
mlflow-skinny                      3.1.0


In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [6]:
# Data 
DATA_DIR = "data"
FILENAME = "raw_data.csv"
REPORTS_DIR = 'reports'

# MLFlow
MLFLOW_TRACKING_URI = "http://localhost:5000"

## Load Data

More information about the dataset can be found in UCI machine learning repository: https://archive.ics.uci.edu/ml/datasets/bike+sharing+dataset

Acknowledgement: Fanaee-T, Hadi, and Gama, Joao, 'Event labeling combining ensemble detectors and background knowledge', Progress in Artificial Intelligence (2013): pp. 1-15, Springer Berlin Heidelberg

In [7]:
# Download original dataset with: python src/load_data.py 
raw_data = pd.read_csv(f"../{DATA_DIR}/{FILENAME}")

# Set datetime index 
raw_data = raw_data.set_index('dteday')

raw_data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
dteday,,,,,,,,,,,,,,,,
2011-01-01,1,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
2011-01-01,2,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2011-01-01,3,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
2011-01-01,4,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
2011-01-01,5,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [8]:
# Define dates for train data
train_dates = ('2011-01-02 00:00:00','2011-03-06 23:00:00')

# Define dates for inference batches
prediction_batches = [ 
    ('2011-03-07 00:00:00','2011-03-13 23:00:00'),
    ('2011-03-14 00:00:00','2011-03-20 23:00:00'),
    ('2011-03-21 00:00:00','2011-03-27 23:00:00'), 
]

## Define column mapping

In [9]:
target = 'cnt'
prediction = 'prediction'
datetime = 'dteday'
numerical_features = ['temp', 'atemp', 'hum', 'windspeed', 'mnth', 'hr', 'weekday']
categorical_features = ['season', 'holiday', 'workingday', ]
FEATURE_COLUMNS = numerical_features + categorical_features

column_mapping = ColumnMapping()
column_mapping.target = target
column_mapping.prediction = prediction
column_mapping.datetime = datetime
column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features

NameError: name 'ColumnMapping' is not defined

# Train a Model

## Train a model

In [ ]:
sample_data = raw_data.loc['2011-01-01 00:00:00':'2011-01-28 23:00:00'].reset_index()

print(sample_data.shape)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(
    sample_data[numerical_features + categorical_features],
    sample_data[target],
    test_size=0.3
)

regressor = ensemble.RandomForestRegressor(random_state = 0, n_estimators = 50)
regressor.fit(X_train, y_train) 

regressor

In [ ]:
model_path = Path('../models/model.joblib')
joblib.dump(regressor, model_path)

# Monitor Model

## Define the reference dataset

In [ ]:
# Define the reference dataset
reference_data = raw_data.loc[train_dates[0]:train_dates[1]]
reference_data['prediction'] = regressor.predict(reference_data[FEATURE_COLUMNS])
reference_data = reference_data.reset_index(drop=True)

print(reference_data.shape)

## Week 1

In [ ]:
current_dates = prediction_batches[0]
current_data = raw_data.loc[current_dates[0]:current_dates[1]]  

print(current_data.shape)
# current_data.head()

In [ ]:
current_prediction = regressor.predict(current_data[numerical_features + categorical_features])
current_data['prediction'] = current_prediction
current_data = current_data.reset_index(drop=True)

print(current_data.shape)

In [ ]:
# Build the Model Monitoring report
model_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])
model_report.run(
    reference_data=reference_data,
    current_data=current_data,
    column_mapping=column_mapping
)


In [ ]:
model_report.show(mode='inline')

### Retrieve Monitoring Metrics

In [ ]:
def get_model_monitoring_metrics(
    regression_quality_report: Report
) -> Dict:

    metrics = {} 
    report_dict = regression_quality_report.as_dict()
    
    metrics['me'] = report_dict['metrics'][0]['result']['current']['mean_error']
    metrics['mae'] = report_dict['metrics'][0]['result']['current']["mean_abs_error"]
    metrics['rmse'] = report_dict['metrics'][0]['result']['current']["rmse"]
    metrics['mape'] = report_dict['metrics'][0]['result']['current']["mean_abs_perc_error"]
    
    return metrics

In [ ]:

model_metrics = get_model_monitoring_metrics(model_report)
model_metrics

## Week 2

In [ ]:
current_dates = prediction_batches[1]
current_data = raw_data.loc[current_dates[0]:current_dates[1]]  

current_prediction = regressor.predict(current_data[numerical_features + categorical_features])
current_data['prediction'] = current_prediction
current_data = current_data.reset_index(drop=True)

print(current_dates)
print(current_data.shape)
# current_data.head()

In [ ]:
# Build the Model Monitoring report
model_report = Report(metrics=[
    RegressionQualityMetric(),
    RegressionErrorPlot(),
    RegressionErrorDistribution()
])
model_report.run(
    reference_data=reference_data,
    current_data=current_data,
    column_mapping=column_mapping
)

model_metrics = get_model_monitoring_metrics(model_report)
model_metrics

# Model Quality Evaluation

In [ ]:
# Set up MLFlow Client
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
print(f"Client tracking uri: {client.tracking_uri}")

# Set experiment name
mlflow.set_experiment("Monitor Model")

In [ ]:

# Run model monitoring for each batch of dates
for current_dates in prediction_batches:
    
    print(f"Current batch dates: {current_dates}") 

    # Start a new Run for the batch
    with mlflow.start_run(run_name=current_dates[1]) as run: 
        
        # Show newly created run metadata info
        print("Experiment id: {}".format(run.info.experiment_id))
        print("Run id: {}".format(run.info.run_id))
        print("Run name: {}".format(run.info.run_name))
            
        # Log parameters
        mlflow.log_param("begin", current_dates[0])
        mlflow.log_param("end", current_dates[1])
        
        # Make predictions for the current batch data
        current_data = raw_data.loc[current_dates[0]:current_dates[1]]
        current_prediction = regressor.predict(current_data[FEATURE_COLUMNS])
        current_data['prediction'] = current_prediction
        current_data = current_data.reset_index(drop=True)

        # Build the Model Monitoring report
        model_report = Report(metrics=[
            RegressionQualityMetric(),
            RegressionErrorPlot(),
            RegressionErrorDistribution()
        ])
        model_report.run(
            reference_data=reference_data,
            current_data=current_data,
            column_mapping=column_mapping
        )
        
        # Log Metrics
        model_metrics = get_model_monitoring_metrics(model_report)
        mlflow.log_metrics(model_metrics)
        
        # Log Monitoring Report 
        monitoring_report_path = f"../{REPORTS_DIR}/model_monitoring_report.html"
        model_report.save_html(monitoring_report_path)
        mlflow.log_artifact(monitoring_report_path)
        
        print(run.info)

In [ ]:
model_report.show(mode='inline')